# Resources and references

This exercise has focused on building a functional(not sequential) 1D conv model, compiling it, saving its weights and saving its layers as image. 

# IMPORTS

In [0]:
#imports
from __future__ import print_function, absolute_import, division

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.utils import plot_model

tf.logging.set_verbosity(tf.logging.INFO)

import json
import pprint

# GOOGLE DRIVE MOUNT

In [0]:
# MOUNT google drive for save and load data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/deep_learning/images_from_examples"

conv1d_model.png  model_conv1d_model.png


# FOLDER VARIABLE NAMES

In [0]:
image_folder = "/content/drive/My Drive/deep_learning/images_from_examples/"
model_folder = "/content/drive/My Drive/deep_learning/saved_models_from_examples/"

# Conv Model Construction Function



*   Conv1d
*   MaxPool1d
*   Conv1D
*   MaxPool1d
*   Drop
*   flatten
*   Dense
*   Drop
*   activation
*   dense





In [0]:
def build_conv1d_model(tensor_input_length, filters=64, kernel_size=3, pooling_size=2, dropout=0.5):
  #todo: maybe add strides
  
  # initialize the placeholdder
  placeholder_input = tf.keras.Input(shape=(tensor_input_length, 1))
  
  #------------- FIRST CNN BLOCK: CONV1D & MAXPOOLING ------------- 
  model = layers.Conv1D(filters=filters, 
                        kernel_size=kernel_size, 
                        padding="same",
                        input_shape=(tensor_input_length, 1),
                        batch_input_shape=(None, tensor_input_length, 1),
                        activation='relu')(placeholder_input)
  
  model = layers.MaxPool1D(pool_size=pooling_size, 
                           padding="same")(model)
  
  # CONSIDER:  model = layers.BatchNormalization(model)  
  # ------------- Second CNN BLOCK: CONV1D & MAXPOOLING ------------- 
  model = layers.Conv1D(filters=filters, 
                        kernel_size=kernel_size, 
                        padding="same", 
                        activation='relu')(model)
  
  model = layers.MaxPool1D(pool_size=pooling_size, 
                           padding="same")(model)
  # ------------- DROPOUT & FLATTEN ------------- 
  model = layers.Dropout(dropout)(model)
  model = layers.Flatten()(model)
  
  
  model = layers.Dense(32, activation='relu')(model)
  model = layers.Dropout(dropout)(model)
  # not sure if this necessary 1
  #model = layers.Activation(activation='relu')(model)
  
  prediction = layers.Dense(1, activation='sigmoid')(model)
  
  model = tf.keras.Model(inputs=placeholder_input, outputs=prediction)
  return model
  # not sure if this necessary 1
#  model = layers.Activation(activation='sigmoid')(model)

# Usage of the constructed model

In [0]:
# placeholder
#inputs = tf.keras.Input(shape=(32,32),dtype=np.float32)
model = build_conv1d_model(tensor_input_length=128)

# print and plot the model
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 128, 1)            0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 128, 64)           256       
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 64, 64)            0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 64, 64)            12352     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 32, 64)            0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 64)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2048)              0         
__________

# Save constructed model image

In [0]:
plot_model(model, to_file=image_folder+"conv1d_model.png")

# Compile and save the model

In [0]:
# todo: choose learning rat
# todo: choose regularization on layers
# todo: choose weight initialization
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
             loss=tf.keras.losses.mean_squared_error,
             metrics=['accuracy'])


#save weights to a checekpoint file
model.save_weights(model_folder+"conv1d_example_weights")

#save weights to a HDF5 file
model.save_weights(model_folder+"conv1d_example_weights.h5", save_format='h5')

# can be saved via json
# jsonModel = model.to_json()
# pprint.pprint(json.loads(jsonModel))

